# Character-based LSTM: Training

In [1]:
import sys
sys.path.append('..')

import os

import numpy as np

from keras.callbacks import ModelCheckpoint, EarlyStopping

from src import models, preprocessing, utils

Using TensorFlow backend.


In [2]:
# Some useful parameters
KEY = 'character_lstm'
DIR = os.path.join(utils.MODELS_DIR, KEY)
units = 512
window_size = 40  # number of characters

### Load data

In [3]:
sonnets = utils.load_shakespeare()

### Pre-processing

In [4]:
encoding, encoded_sonnets = preprocessing.encode_characters_onehot(sonnets)
n_chars = len(encoding)

In [5]:
# Generate training data, where each character is an `n_chars`-dimensional vector.
X = []
Y = []
for sonnet in encoded_sonnets:
    x = np.zeros((len(sonnet) - window_size, window_size, n_chars))
    y = np.zeros((len(sonnet) - window_size, n_chars))
    
    for i in range(len(sonnet) - window_size):
        x[i] = sonnet[i:i+window_size]
        y[i] = sonnet[i+window_size]
    X.append(x)
    Y.append(y)
X = np.vstack(X)
Y = np.vstack(Y)

### Define model

In [6]:
# Save model parameters.
params = {
    'encoding': encoding,
    'window_size': window_size,
    'units': units
}
utils.save_pickle(params, os.path.join(DIR, 'params.pkl'))

In [7]:
model = models.CharacterLSTM(units, window_size, n_chars)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Train

In [9]:
# Save model with best accuracy.
save_path = os.path.join(DIR, 'model-{epoch:03d}-{loss:.4f}.h5')
checkpoint = ModelCheckpoint(save_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
# Early stopping condition. Stop when loss has stopped decreasing for 10 epochs.
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)
model.fit(X, Y, epochs=1000, batch_size=64, callbacks=[checkpoint, es])

Epoch 1/1000
87360/87360 [==============================] - 46s 529us/step - loss: 2.3306 - accuracy: 0.3266

Epoch 00001: loss improved from inf to 2.33064, saving model to C:\Users\Joseph\Documents\GitHub\cs155-miniproject3\models\character_lstm\model-001-2.3306.h5
Epoch 2/1000
87360/87360 [==============================] - 46s 525us/step - loss: 1.8745 - accuracy: 0.4394

Epoch 00002: loss improved from 2.33064 to 1.87451, saving model to C:\Users\Joseph\Documents\GitHub\cs155-miniproject3\models\character_lstm\model-002-1.8745.h5
Epoch 3/1000
87360/87360 [==============================] - 46s 525us/step - loss: 1.6970 - accuracy: 0.4824

Epoch 00003: loss improved from 1.87451 to 1.69700, saving model to C:\Users\Joseph\Documents\GitHub\cs155-miniproject3\models\character_lstm\model-003-1.6970.h5
Epoch 4/1000
87360/87360 [==============================] - 46s 525us/step - loss: 1.5800 - accuracy: 0.5135

Epoch 00004: loss improved from 1.69700 to 1.58000, saving model to C:\Users\J